In [ ]:
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

import os
import keras
import h5py
import librosa
import itertools
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Dropout
from keras.models import Model,load_model
from keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
artists = {"Arjit_Singh":0,"Armaan":1,"Atif Aslam":2,"Sonu Nigam":3}

In [ ]:
X = np.load("/kaggle/input/specdata/Data/x_singers_npy.npy")
y = np.load("/kaggle/input/specdata/Data/y_singers_npy.npy")

In [ ]:
y.shape

In [ ]:
X.shape


In [ ]:
for i in range(4):
    print("No of mel Spectograms for Music Artist",list(artists.keys())[i],"-->" , list(y).count(i))

In [ ]:
# One hot encoding of the labels
y = to_categorical(y)

In [ ]:
y.shape

In [ ]:
X_stack = np.squeeze(np.stack((X,) * 3, -1))
X_stack.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_stack, y, test_size=0.3, random_state=42, stratify = y)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
values, count = np.unique(np.argmax(y_train, axis=1), return_counts=True)
plt.bar(values, count)

values, count = np.unique(np.argmax(y_test, axis=1), return_counts=True)
plt.bar(values, count)
plt.show()

In [ ]:
input_shape = X_train[0].shape
output_shape = 4

In [ ]:
input_tensor = Input(shape=input_shape)
vgg16 = VGG16(include_top=False, weights='imagenet',input_shape = input_shape)
top = Sequential()
top.add(Flatten(input_shape=vgg16.output_shape[1:]))
top.add(Dense(256, activation='relu'))
top.add(Dropout(0.5))
top.add(Dense(output_shape, activation='softmax'))
print(vgg16.output_shape[1:])
print(vgg16.input)
print(vgg16.output)
print(top(vgg16.output))
model = Model(inputs=vgg16.input, outputs=top(vgg16.output))


In [ ]:
model.summary()

In [ ]:
for i, layer in enumerate(model.layers):
        
        if(i <= 5):
            layer.trainable = False
        else:
            layer.trainable = True

In [ ]:
for layer in model.layers :
   print(layer,"----->",layer.trainable)

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=4)

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train,
          batch_size=128,
          epochs=30,
          verbose=1,
          validation_data=(X_test, y_test),
          shuffle=True,
          callbacks=[early_stopping_callback])

In [ ]:
model.save

In [ ]:
import os
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink
FileLink('model.pth')

In [ ]:
from keras.models import load_model

In [ ]:
new  = load_model("/kaggle/input/trainedmodel/model1.h5")

In [ ]:
for layer in new.layers :
   print(layer,"----->",layer.trainable)

In [ ]:
p = new.predict(X_test)

In [ ]:
type(p)

In [ ]:
p.shape

In [ ]:
p[0]

In [ ]:
k = p.argmax(axis=1)

In [ ]:
k.shape

In [ ]:
mat = confusion_matrix(y_test.argmax(axis =1),k)

In [ ]:
list(artists.keys())

In [ ]:
import seaborn as sns
import pandas as pd
df = pd.DataFrame(mat,columns = list(artists.keys()),index=list(artists.keys()))
print(df)
sns.heatmap(mat,annot = True , cmap='Blues')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report( y_test.argmax(axis =1), k,target_names= list(artists.keys())))